In [2]:
import sys
import torch

sys.path.append('./Implementation')
from v1 import Game

ModuleNotFoundError: No module named 'v1'

In [15]:
# RANDOM BOT # 

import random
import math

class QuoridorBot:
    def __init__(self, game, player_id):
        self.game = game
        self.player_id = player_id
        self.bot = self.game.players[self.player_id]

    def make_move(self):
        # Check if it's the bot's turn
        if self.game.cur_player != self.player_id:
            return False
    
        done = False
        while not done:
            if random.choice(["move", "wall"]) == "move":
                done = self.make_move_decision()
            else:
                done = self.make_wall_decision()
        return done

    def make_move_decision(self):
        # Example: Move towards the goal
        directions = [(-1, 0), (1, 0), (0, 1), (0, -1), (-1, 1), (-1,-1), (1, -1), (1, 1), (2,0), (-2,0), (0,2), (0,-2)]  # Possible directions
        direction = random.choice(directions)
        return self.game.perform_action(self.bot, "move", direction)

    def make_wall_decision(self):
        # Example: Place a wall randomly
        wall_position = (math.ceil(8*random.uniform(0, 1))+0.5, math.ceil(8*random.uniform(0, 1))+0.5)
        wall_orientation = random.choice(["horizontal", "vertical"])
        return self.game.perform_action(self.bot, "wall", (wall_position, wall_orientation))

In [17]:
game = Game()
bot = QuoridorBot(game, 1)
p0, p1 = game.players
while game.winner == None:
    action = input("Player move").split(";")
    action_type = action[0]
    action = action[1]
    game.perform_action(p0, "move", (0,1))
    bot.make_move()


Moved player 0 to position (5,2)
horizontal wall at (6.5,1.5) placed successfully.
Moved player 0 to position (5,3)
horizontal wall at (5.5,6.5) placed successfully.
Moved player 0 to position (5,4)
horizontal wall at (7.5,5.5) placed successfully.
Moved player 0 to position (5,5)
horizontal wall at (3.5,6.5) placed successfully.
Moved player 0 to position (5,6)
vertical wall at (6.5,6.5) placed successfully.
Invalid move for player 0
Invalid move for player 0
Invalid move for player 0
Invalid move for player 0


KeyboardInterrupt: Interrupted by user

In [1]:
from copy import deepcopy

# MINIMAX BOT # 

import random
import math

class MiniMaxBot:
    def __init__(self, game, player_id):
        self.game = game
        self.player_id = player_id
        self.bot = self.game.players[self.player_id]
        self.player = self.game.players[1 - self.player_id]

    def make_move(self):
        # Check if it's the bot's turn
        if self.game.cur_player != self.player_id:
            return False

        max_min_h = None
        best_move = None
        for move in self.generate_all_legal_moves(game, bot):
            game_copy = deepcopy(self.game)
            bot_copy = game_copy.players[self.player_id]
            player_copy = game_copy.players[1 - self.player_id]
            game_copy.perform_action(bot_copy, move[0], move[1])
            min_h = None
            for response in self.generate_all_legal_moves(game_copy, player_copy):
                game_copy.perform_action(player_copy, response[0], response[1])
                x = self.heuristic(game_copy, bot_copy, player_copy)
                if min_h == None or min_h < x:
                    min_h = x
            if max_min_h == None or (max_min_h < min_h):
                best_move = move
        
        self.game.perform_action(bot, best_move[0], best_move[1])
                



    
    def generate_all_legal_moves(self, game, player):
        directions = [(-1, 0), (1, 0), (0, 1), (0, -1), (-1, 1), (-1,-1), (1, -1), (1, 1), (2,0), (-2,0), (0,2), (0,-2)]  # Possible directions
        walls_pos = [(i+1.5, j+1.5) for i in range(8) for j in range(8)]
        for dir in directions:
            if game.is_legal_move(player, dir):
                yield "move", dir

        for wall_pos in walls_pos:
            if game.is_legal_wall(player, wall_pos, "vertical", undo_successful_wall = True):
                yield "wall", (wall_pos, "vertical")
            if game.is_legal_wall(player, wall_pos, "horizontal", undo_successful_wall = True):
                yield "wall", (wall_pos, "horizontal")

    def heuristic(self, game, bot, player):
        return game.check_path_to_end(bot)[1] - game.check_path_to_end(player)[1]